Wahlpflichtfach Künstliche Intelligenz II: Praktikum 

---

# 01 - Computer Vision: Convolutional Neural Networks (CNNs)

Im Folgenden sollen Sie ein CNN in TensorFlow bauen. Als Datensatz verwenden wir dafür die [CIFAR-Bilder](https://www.cs.toronto.edu/~kriz/cifar.html)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%load_ext tensorboard
%matplotlib inline

In [ ]:
from keras.datasets.cifar10 import load_data

(train_images, train_labels), (test_images, test_labels) = load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

class_names = [
    "airplane", 
    "automobile", 
    "bird", 
    "cat", 
    "deer",
    "dog", 
    "frog", 
    "horse", 
    "ship", 
    "truck"
]

## Data Understanding

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(10,10))
for i in range(5):
    for j in range(5):
        axes[i, j].imshow(train_images[i*5+j], cmap=plt.cm.binary)
        axes[i, j].set_title(class_names[train_labels[i*5+j][0]])
        axes[i, j].axis('off')
fig.show()

In [ ]:
print(f"Es gibt {len(train_images)} Trainings-Bilder, die eine Größe von {train_images[0].shape} haben.")
print(f"Es gibt {len(test_images)} Test-Bilder.")
print(f"Die Labels sind: {set(test_labels.flatten())}")

In [ ]:
train_labels = tf.squeeze(tf.one_hot(train_labels, 10))
test_labels = tf.squeeze(tf.one_hot(test_labels, 10))

## Das erste CNN erstellen

Bevor wir ein CNN erstellen, sollten sie sich anschauen was ein [Convolutional Layer](https://developers.google.com/machine-learning/glossary?hl=de#convolutional_layer) und ein [Pooling Layer](https://developers.google.com/machine-learning/glossary?hl=de#pooling) sind und wie diese funktionieren.

Im Folgenden erstellen wir unser erstes CNN schauen sie sich in ruhe dazu die [Dokumentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers) der Layer die benötigt werden an.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

cnn_1 = Sequential(name='cnn_1')
# Feature extractor

cnn_1.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
cnn_1.add(MaxPooling2D((2, 2)))
cnn_1.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn_1.add(MaxPooling2D((2, 2)))
cnn_1.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
# Classifier
cnn_1.add(Flatten())
cnn_1.add(Dense(64, activation='relu'))
cnn_1.add(Dense(10, activation='softmax'))

## Training des CNNs

Das CNN können sie genauso wie alle anderen Modelle in Tensorflow mit `compile()` Kompilieren und mit `fit()` trainieren

In [ ]:
cnn_1.summary()

In [ ]:
from datetime import datetime
from keras.callbacks import TensorBoard
from keras.losses import CategoricalCrossentropy

log_dir = "../../logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

cnn_1.compile(
    optimizer='adam',
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)

cnn_1.fit(
    train_images, 
    train_labels, 
    batch_size=64, 
    epochs=10, 
    validation_data=(test_images, test_labels),
    callbacks=[tensorboard_callback]
)

## Tensorboard

In [ ]:
%tensorboard --logdir logs/fit

## Performance überprüfen
Validieren des Models mit `evaluate()` ist ebenfalls möglich

In [ ]:
test_loss, test_acc = cnn_1.evaluate(test_images,  test_labels, verbose=2)
print(f"Die Test-Accuracy beträgt {test_acc}")

In [ ]:
predictions = cnn_1.predict(np.expand_dims(test_images[0], axis=0))

y_pred = np.argmax(predictions, axis=1)[0]
y_true = np.argmax(test_labels[0])
print(f"predicted label: {class_names[y_pred]} ({y_pred})")
print(f"actual label: {class_names[y_true]} ({y_true})")

plt.imshow(train_images[0], cmap=plt.cm.binary)

## Möglichkeiten zur Verbesserung

Auch hier können die Hyperparameter optimiert werden, um eine Besser Performance zu erzeugen:
- Mehr oder weniger Schichten im Feature Extractor
- Andere Kernel Size, Stride Size oder Pooling Size
- Mehr oder weniger Kernel in den Convolutional-Schichten

Im Folgenden sehen sie ein anderes Beispiel. Optimierung lassen sich ebenfalls mit den vorgestellten suchverfahren mit dem Keras-Tuner erledigen.

In [ ]:
from keras.layers import AveragePooling2D, GlobalAveragePooling2D   

cnn_2 = Sequential(name='CNN_2')
# Feature extractor
cnn_2.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
cnn_2.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
cnn_2.add(AveragePooling2D((2, 2)))
cnn_2.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
cnn_2.add(Conv2D(128, (7, 7), activation='relu', padding='same'))
# Classifier
cnn_2.add(GlobalAveragePooling2D())
cnn_2.add(Dense(256, activation='relu'))
cnn_2.add(Dense(10, activation='softmax'))

In [ ]:
cnn_2.summary()

In [ ]:
log_dir = "../../logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

cnn_2.compile(
    optimizer='adam',
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)

cnn_2.fit(
    train_images, 
    train_labels, 
    batch_size=64, 
    epochs=10, 
    validation_data=(test_images, test_labels),
    callbacks=[tensorboard_callback]
)

In [ ]:
test_loss, test_acc = cnn_2.evaluate(test_images, test_labels, verbose=2)
print(f"Die Test-Accuracy beträgt {test_acc}")

## Weiterführende Themen

In der Einheit wurden die Grundlagen von CNNs beleuchtet. Zu dieses Thema gibt jedoch noch viele weitere Themen. Die Bibliothek Keras hat mit [KerasCV](https://keras.io/keras_cv/) ein eigenes Segment in den Lösungen für die Datenaufbereitung oder [vortrainierte Modelle](https://keras.io/api/keras_cv/models/) bereitgestellt werden.

Zusätzlich gibt es auch mit dem [Transformer](https://huggingface.co/docs/transformers/model_doc/vit) Ansatz CV Lösungen.


---

Wahlpflichtach Künstliche Intelligenz II: Praktikum 